In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
wind_turbines = pd.read_csv('wind_turbines.csv')

In [5]:
wind_turbines.isnull().sum()

case_id                                         0
faa_ors                                      5792
faa_asn                                      5122
usgs_pr_id                                  32545
plant_id                                     5793
turbine_state                                   0
turbine_county                                  0
turbine_fips                                    0
project_name                                    0
project_year                                  613
qty_turbines_per_project                        0
project_mw_capacity                          4482
turbine_manufacturer                         5640
turbine_model                                5777
turbine_output_power                         5480
turbine_height(m)                            6180
turbine_rotor_diameter(m)                    5934
turbine_rotor_swept_area(m squared)          5934
turbine_total_height(m)                      6180
retrofitted?                                    0


In [23]:
wind_turbines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70808 entries, 0 to 70807
Data columns (total 27 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   case_id                                   70808 non-null  int64  
 1   faa_ors                                   65016 non-null  object 
 2   faa_asn                                   65686 non-null  object 
 3   usgs_pr_id                                38263 non-null  float64
 4   plant_id                                  65015 non-null  float64
 5   turbine_state                             70808 non-null  object 
 6   turbine_county                            70808 non-null  object 
 7   turbine_fips                              70808 non-null  int64  
 8   project_name                              70808 non-null  object 
 9   project_year                              70195 non-null  float64
 10  qty_turbines_per_project          

In [7]:
turbine_operators = pd.read_csv('turbine_operators.csv')

In [8]:
turbine_operators.isnull().sum()

plant_id                       0
combo_heat_and_power_plant?    0
plant_name                     1
operator_name                  1
operator_id                    1
                              ..
elec_fuel_consump_qty          0
tot_fuel_consump_mmbtu         0
tot_elec_fuel_consump_mmbtu    0
tot_net_mwh_gen_mmbtu          0
year                           0
Length: 95, dtype: int64

In [21]:
# join dataframes
us_operators = pd.merge(turbine_operators, wind_turbines, on='plant_id')

# filter fuel type to just wind
us_operators = us_operators[us_operators['reported_fuel_type_code'].str.lower() == 'wnd']

# group columns
us_operators = us_operators.groupby(['operator_name', 'turbine_state', 'turbine_manufacturer']).agg(
    sum_of_turbines=('qty_turbines_per_project', 'sum'),
    net_evergy_generation=('tot_net_mwh_gen_mmbtu', 'sum')
)

# sort by energy generated
us_operators = us_operators.sort_values(by='net_evergy_generation', ascending=False)

In [22]:
us_operators

sum_of_turbines  \
operator_name                 turbine_state turbine_manufacturer                    
MidAmerican Energy Co         IA            Vestas                          85934   
                                            Siemens                        134777   
Capricorn Ridge Wind LLC      TX            GE Wind                         69354   
Public Service Co of Colorado CO            Vestas                         101784   
FPL Energy Horse Hollow LLC   TX            GE Wind                         42081   
...                                                                           ...   
Harbor Wind Tenant LLC        TX            Guodian                            36   
Kingston Wind Independence    MA            Gamesa                              9   
Rock County Energy Center LLC MN            NEG Micon                          49   
Performance Services          IN            Nordic                              2   
Tennessee Valley Authority    TN            Vestas                              9   

                                                                  net_evergy_generation  
operator_name                 turbine_state turbine_manufacturer                         
MidAmerican Energy Co         IA            Vestas                         1.004841e+09  
                                            Siemens                        9.674847e+08  
Capricorn Ridge Wind LLC      TX            GE Wind                        7.727186e+08  
Public Service Co of Colorado CO            Vestas                         7.405065e+08  
FPL Energy Horse Hollow LLC   TX            GE Wind                        7.314160e+08  
...                                                                                 ...  
Harbor Wind Tenant LLC        TX            Guodian                        0.000000e+00  
Kingston Wind Independence    MA            Gamesa                         0.000000e+00  
Rock County Energy Center LLC MN            NEG Micon                      0.000000e+00  
Performance Services          IN            Nordic                         0.000000e+00  
Tennessee Valley Authority    TN            Vestas                         0.000000e+00  

[947 rows x 2 columns]

In [28]:
# join dataframes
energy_per_operator = pd.merge(turbine_operators, wind_turbines, on='plant_id')

# filter fuel type to just wind
energy_per_operator = energy_per_operator[energy_per_operator['reported_fuel_type_code'].str.lower() == 'wnd']

# group columns
energy_per_operator = energy_per_operator.groupby('operator_name')['tot_net_mwh_gen_mmbtu'].sum().reset_index()

# sort by energy generated
energy_per_operator = energy_per_operator.sort_values(by='tot_net_mwh_gen_mmbtu', ascending=False)

# rename column
energy_per_operator = energy_per_operator.rename(columns={'tot_net_mwh_gen_mmbtu': 'net_evergy_generation'})

In [29]:
energy_per_operator

,operator_name,net_evergy_generation
370,MidAmerican Energy Co,2.639545e+09
30,Avangrid Renewables LLC,1.654197e+09
469,RWE Renewables Americas LLC,1.446312e+09
162,EDF Renewable Asset Holdings Inc.,1.252402e+09
0,AE Power Services LLC,1.166239e+09
...,...,...
489,Rock County Energy Center LLC,0.000000e+00
527,Southwestern Public Service Co,0.000000e+00
317,Kingston Wind Independence,0.000000e+00
447,Performance Services,0.000000e+00


In [ ]:
# join dataframes
df = pd.merge(wind_turbines, geocodes, left_on='turbine_fips', right_on='state_county_fips')

# adding pop data
df = df[['wind_turbines.*', 'population']]

In [ ]:
# count of turbines and population by county
pop_and_turbines = wind_turbines_new.groupby(['t_county', 't_state', 'population'])['p_tnum'].count().reset_index()

# rename columns
pop_and_turbines = pop_and_turbines.rename(columns={'t_county': 'county', 't_state': 'state', 'p_tnum': 'number of turbines'})

# sorting
pop_and_turbines = pop_and_turbines.sort_values(by='number of turbines', ascending=False)

# dropping dups
pop_and_turbines = pop_and_turbines.drop_duplicates(subset=['county', 'state'])

# reset index
pop_and_turbines = pop_and_turbines.reset_index(drop=True)